Group: Austin Wang, Joe Higgins, Lawrence Moore

# Question 1

# SOCP

In [2]:
# Import necessary modules
import cvxpy as cvx
import numpy as np
from scipy import random
from scipy import linalg

In [3]:
def generate_points(num, dim):
    return np.matrix(4 * np.random.random((num, dim)) - 2)

In [5]:
num_sensors = 10
num_anchors = 4
dim = 2
range_mult = 1

anchors = generate_points(num_anchors, dim) * range_mult
sensors = generate_points(num_sensors, dim)

d_sa = list(map(lambda s: list(map(lambda a: np.linalg.norm(a - s), anchors)), sensors))

d_ss = list(map(lambda s1: 
        list(map(lambda s2: np.linalg.norm(s1 - s2), sensors))
    , sensors))

In [6]:
x = cvx.Variable(num_sensors, dim)
objective = cvx.Minimize(0)

In [7]:
constraints  = []
for i in range(num_sensors):
    x_i = x[i, :]
    for j in range(num_anchors):
        constraints.append(cvx.norm(x_i - anchors[j]) <= d_sa[i][j])
    for j in range(num_sensors):
        if i < j:
            constraints.append(cvx.norm(x_i - sensors[j]) <= d_ss[i][j])


In [240]:
print(sensors)

[[-1.42275272  1.6052675 ]
 [ 0.87439427 -0.83121965]
 [-0.35642879  0.70686266]
 [ 1.95111945 -1.3025779 ]
 [-0.32487899  1.16506877]
 [-1.70155146 -0.1441852 ]
 [-0.35626019 -0.26393005]
 [-1.73766309 -0.37360521]
 [ 0.51652128  0.05486113]
 [ 0.55180187 -1.9985776 ]]


In [241]:
print(anchors)

[[ 0.79000745 -0.30830019]
 [-1.5201135   0.96813936]
 [-0.22088681  0.03526299]
 [-0.98710229  0.70550747]]


In [267]:
prob = cvx.Problem(objective, constraints)

# The optimal objective is returned by prob.solve().
result = prob.solve(solver = 'MOSEK')
# The optimal value for x is stored in x.value.
print(x.value)

[[-0.66812887 -0.89937099]
 [-0.64038825 -0.65681491]
 [-0.88529556 -1.25097927]
 [-0.99294347  1.34253601]
 [ 0.41043658 -1.70089718]
 [ 0.16732836  0.56002081]
 [-0.2893331  -1.56912421]
 [ 1.00274363 -0.04602282]
 [-0.16143208 -1.16948561]
 [ 0.5888892  -0.0959116 ]]


In [268]:
print(abs(x.value-sensors)<= 10**-5)

[[ True  True]
 [ True  True]
 [False False]
 [False False]
 [ True  True]
 [ True  True]
 [False False]
 [False False]
 [False False]
 [False False]]


# SDP

In [269]:
Z = cvx.Semidef(num_sensors + dim)

In [8]:
def sum_elem_product(A,B):
    return cvx.sum_entries(cvx.mul_elemwise(A, B))

In [9]:
#Make first set of constraint matrices (look like identity)
def enforce_id(dim, num_sensors):
    matrices = []
    rhs = []
    for i in range(dim):
        new_matrix = np.zeros((dim+num_sensors, dim+num_sensors))
        new_matrix[i,i] = 1
        matrices.append(new_matrix)
        rhs.append(1)
        
    return (matrices, rhs)

#Make second set of constraint matrices (symmetric holders) 
def enforce_id2(dim, num_sensors):
    matrices = []
    rhs = []
    for i in range(dim):
        for j in range(dim):
            new_matrix = np.identity(dim)
            if(j > i):
                new_matrix[i,j] = 1
                new_matrix[j,i] = 1
                big_matrix = np.zeros((dim+num_sensors, dim+num_sensors))
                big_matrix[0:dim,0:dim] = new_matrix
                matrices.append(big_matrix)
                rhs.append(dim)
                
    return (matrices, rhs)


#Make third set of constraint matrices (anchors to sensors)
def sensor_constraints(dim, num_sensors):
    matrices = []
    rhs = []
    zero_vec_dim = np.zeros(dim)
    for i in range(num_sensors):
        for j in range(i+1, num_sensors):
            zero_vec_num_s = np.zeros(num_sensors)
            zero_vec_num_s[i] = 1
            zero_vec_num_s[j] = -1
            
            new_vec = np.matrix(np.append(zero_vec_dim, zero_vec_num_s))
            
            new_matrix = np.dot(np.transpose(new_vec), new_vec)
            
            matrices.append(new_matrix)
            rhs.append(d_ss[i][j]**2)

    return (matrices, rhs)

#Make fourth set of constraint matrices (sensors to sensors)
def anchor_constraints(num_anchors, num_sensors):
    matrices = []
    rhs = []
    for i in range(num_anchors):
        for j in range(num_sensors):
            zero_vec_num_s = np.zeros(num_sensors)
            zero_vec_num_s[j] = -1

            new_vec = np.append(np.array(anchors[i,:]), np.array(zero_vec_num_s))
            new_vec = np.matrix(new_vec)
            new_matrix = np.dot(np.transpose(new_vec), new_vec)
            matrices.append(new_matrix)
            rhs.append(d_sa[j][i]**2)

    return (matrices, rhs)

A = enforce_id(dim, num_sensors)
B = enforce_id2(dim, num_sensors)
C = sensor_constraints(dim, num_sensors)
D = anchor_constraints(num_anchors, num_sensors)

In [272]:
constraints = []
for id_constraint, rhs in zip(A[0], A[1]):
    constraints.append(sum_elem_product(id_constraint, Z) == rhs)
for id_constraint2, rhs in zip(B[0], B[1]):
    constraints.append(sum_elem_product(id_constraint2, Z) == rhs)
for sensor_constraint, rhs in zip(C[0], C[1]):
    constraints.append(sum_elem_product(sensor_constraint, Z) == rhs)
for anchor_constraint, rhs in zip(D[0], D[1]):
    constraints.append(sum_elem_product(anchor_constraint, Z) == rhs)

In [273]:
prob = cvx.Problem(objective, constraints)
result = prob.solve(solver = 'MOSEK')

In [274]:
print("Real sensors: ")
print(sensors)
print("Generated Sensors: ")
generated = np.transpose(Z[0:dim, dim:dim+num_sensors].value)
print(generated)

Real sensors: 
[[-0.66812887 -0.89937099]
 [-0.64038825 -0.65681491]
 [-1.50042933 -1.98543567]
 [-1.10276266  1.51493211]
 [ 0.41043658 -1.70089719]
 [ 0.16732836  0.56002081]
 [-0.39070409 -1.95306535]
 [ 1.74767567  0.35239902]
 [-0.19505899 -1.75562855]
 [ 1.66120014 -1.40530023]]
Generated Sensors: 
[[-0.66812887 -0.89937099]
 [-0.64038825 -0.65681491]
 [-1.50042933 -1.98543567]
 [-1.10276266  1.51493211]
 [ 0.41043658 -1.70089719]
 [ 0.16732836  0.56002081]
 [-0.39070409 -1.95306535]
 [ 1.74767567  0.35239902]
 [-0.19505899 -1.75562855]
 [ 1.66120014 -1.40530023]]


In [251]:
def super_quick_func(real, generated):
    for_real_do_results = []
    for real_sens, real_gen in zip(real, generated):
        for_real_do_results.append(np.linalg.norm(real_sens - real_gen) < 10**(-6))
    return for_real_do_results


In [207]:
super_quick_func(sensors, generated)

[True, True, True, True, True, True, True, True, True, True]

Yo, anchors is huge == it'll localize.

### Part 3 

In [32]:
def grad(X):
    sensor_distance_sum = np.zeros((1,X.shape[1]))
    anchor_distance_sum = np.zeros((1,X.shape[1]))
    gradient = np.zeros(X.shape)

    for i, sensor_i in enumerate(X):
        for j, sensor_j in enumerate(X):
            if(i != j):
                sensor_distance_sum += (np.linalg.norm(sensor_i - sensor_j)**2 - \
                                        np.linalg.norm(sensors[i,:] - sensors[j,:])**2) * \
                                       (sensor_i - sensor_j)

        for k, anchor_k in enumerate(anchors):
            anchor_distance_sum += (np.linalg.norm(anchor_k - sensor_i)**2 - \
                                    np.linalg.norm(anchors[k,:] - sensors[i,:])**2) * \
                                   (sensor_i - anchor_k)

        gradient[i,:] = 8*sensor_distance_sum + 4*anchor_distance_sum

    return gradient

In [33]:
check = 1000
max_iter = 10000
k = 0

# Initial sensors guess
sensors_0 = generate_points(num_sensors, dim)
sensors_k = sensors_0

# D iteration
alpha = .0000001
while check > 10**-8 and k < max_iter:
    sensors_k1 = sensors_k - alpha * grad(sensors_k)
    check = np.linalg.norm(sensors_k1 - sensors_k)
    sensors_k = sensors_k1
    k = k+1
    

In [31]:
sensors

matrix([[-0.82963175,  1.49491793],
        [-1.6397261 , -0.88638492],
        [-0.57167572, -1.90004467],
        [-0.57647951, -1.41087409],
        [ 0.12781571, -0.32245923],
        [-1.03637545, -0.24204694],
        [-1.55879562,  1.60678321],
        [ 1.34062095,  1.62617786],
        [ 0.88287445, -1.71321207],
        [ 1.62936614,  0.73302684]])

In [34]:
sensors_k

matrix([[-0.82963175,  1.49491793],
        [-1.6397261 , -0.88638492],
        [-0.57167572, -1.90004467],
        [-0.57647952, -1.41087409],
        [ 0.12781572, -0.32245924],
        [-1.03637549, -0.24204693],
        [-1.55879554,  1.60678318],
        [ 1.34062092,  1.62617787],
        [ 0.88287422, -1.71321199],
        [ 1.62936685,  0.73302657]])

### Question 2

In [36]:
Z = cvx.Semidef(num_sensors + dim)

In [37]:
#order of indexing: (1,2), (1,3), (1,4)... (9,10)
delta_p_ss = cvx.Variable((int) (num_sensors*(num_sensors-1)/2))
delta_pp_ss = cvx.Variable((int) (num_sensors*(num_sensors-1)/2))
                           
#order of indexing: (anchor, sensor), primary sort on anchors
delta_p_sa = cvx.Variable(num_anchors * num_sensors)
delta_pp_sa = cvx.Variable(num_anchors * num_sensors)

In [38]:
constraints = []

constraints.append(delta_p_ss >= 0)
constraints.append(delta_pp_ss >= 0)
constraints.append(delta_p_sa >= 0)
constraints.append(delta_pp_sa >= 0)

for id_constraint, rhs in zip(A[0], A[1]):
    constraints.append(sum_elem_product(id_constraint, Z) == rhs)
for id_constraint2, rhs in zip(B[0], B[1]):
    constraints.append(sum_elem_product(id_constraint2, Z) == rhs)
for sensor_constraint, rhs, dp, dpp in zip(C[0], C[1], delta_p_ss, delta_pp_ss):
    constraints.append(sum_elem_product(sensor_constraint, Z) + \
                       dp - dpp == rhs)
for anchor_constraint, rhs, dp, dpp in zip(D[0], D[1], delta_p_sa, delta_pp_sa):
    constraints.append(sum_elem_product(anchor_constraint, Z) + \
                       dp - dpp == rhs)


In [39]:
objective = cvx.Minimize(cvx.sum_entries(delta_p_ss) + \
                         cvx.sum_entries(delta_pp_ss) + \
                         cvx.sum_entries(delta_p_sa) + \
                         cvx.sum_entries(delta_pp_sa))

In [40]:
prob = cvx.Problem(objective, constraints)
result = prob.solve(solver = 'MOSEK')

In [41]:
print("Real sensors: ")
print(sensors)
print("Generated Sensors: ")
generated = np.transpose(Z[0:dim, dim:dim+num_sensors].value)
print(generated)

Real sensors: 
[[ 1.54156573 -1.76389547]
 [-1.05057748  1.03173858]
 [ 1.98800959  1.01284084]
 [-1.45969671 -1.38504371]
 [-1.77460924  1.13546463]
 [ 0.63673365 -0.58291593]
 [-0.88334992 -1.69480079]
 [ 0.50113216 -0.31842127]
 [-1.25266142 -0.27797936]
 [-1.51687772 -0.96143237]]
Generated Sensors: 
[[ 1.54156573 -1.76389547]
 [-1.05057748  1.03173858]
 [ 1.98800959  1.01284084]
 [-1.45969671 -1.38504371]
 [-1.77460924  1.13546463]
 [ 0.63673365 -0.58291593]
 [-0.88334992 -1.69480079]
 [ 0.50113216 -0.31842127]
 [-1.25266142 -0.27797936]
 [-1.51687772 -0.96143237]]


In [42]:
check = 1000
max_iter = 10000
k = 0

# Initial sensors guess
sensors_0 = generated
sensors_k = sensors_0

# D iteration
alpha = .000001
while check > 10**-8 and k < max_iter:
    sensors_k1 = sensors_k - alpha * grad(sensors_k)
    check = np.linalg.norm(sensors_k1 - sensors_k)
    sensors_k = sensors_k1
    k = k+1

In [43]:
print("Real sensors: ")
print(sensors)
print("Generated Sensors: ")
generated = np.transpose(Z[0:dim, dim:dim+num_sensors].value)
print(generated)

Real sensors: 
[[ 1.54156573 -1.76389547]
 [-1.05057748  1.03173858]
 [ 1.98800959  1.01284084]
 [-1.45969671 -1.38504371]
 [-1.77460924  1.13546463]
 [ 0.63673365 -0.58291593]
 [-0.88334992 -1.69480079]
 [ 0.50113216 -0.31842127]
 [-1.25266142 -0.27797936]
 [-1.51687772 -0.96143237]]
Generated Sensors: 
[[ 1.54156573 -1.76389547]
 [-1.05057748  1.03173858]
 [ 1.98800959  1.01284084]
 [-1.45969671 -1.38504371]
 [-1.77460924  1.13546463]
 [ 0.63673365 -0.58291593]
 [-0.88334992 -1.69480079]
 [ 0.50113216 -0.31842127]
 [-1.25266142 -0.27797936]
 [-1.51687772 -0.96143237]]


## Question 4

In [24]:
def sum_elem_product_noncvx(A,B):
    return np.sum(np.multiply(A,B))

def make_AX_less_b(X):
    output = []
    for id_constraint, rhs in zip(A[0], A[1]):
        output.append(sum_elem_product_noncvx(id_constraint, X) - rhs)
    for id_constraint2, rhs in zip(B[0], B[1]):
        output.append(sum_elem_product_noncvx(id_constraint2, X) - rhs)
    for sensor_constraint, rhs in zip(C[0], C[1]):
        output.append(sum_elem_product_noncvx(sensor_constraint, X) - rhs)
    for anchor_constraint, rhs in zip(D[0], D[1]):
        output.append(sum_elem_product_noncvx(anchor_constraint, X) - rhs)
    
    return output

def scriptAt_y(y):
    output = np.zeros((dim+num_sensors,dim+num_sensors)) #start sum of y_i*A_i
    i = 0
    for id_constraint, rhs in zip(A[0], A[1]):
        output += np.multiply(id_constraint, y[i])
        i = i + 1
    for id_constraint2, rhs in zip(B[0], B[1]):
        output += np.multiply(id_constraint2, y[i])
        i = i + 1
    for sensor_constraint, rhs in zip(C[0], C[1]):
        output += np.multiply(sensor_constraint, y[i])
        i = i + 1
    for anchor_constraint, rhs in zip(D[0], D[1]):
        output += np.multiply(anchor_constraint, y[i])
        i = i + 1
    return output

def grad(X):
    return scriptAt_y(make_AX_less_b(X))

me = np.zeros((dim+num_sensors,dim+num_sensors))
me[0:dim,0:dim] = np.eye(dim)
me[0:dim, dim:dim+num_sensors] = np.transpose(sensors)
me[dim:dim+num_sensors, 0:dim] = sensors
me[num_sensors:dim+num_sensors,num_sensors:dim+num_sensors] = \
        np.dot(np.transpose(sensors),sensors)

#prepare iterations
thing = random.rand(dim+num_sensors,dim+num_sensors)
X_0 = np.dot(thing,thing.transpose())
#X_0 = np.eye(dim+num_sensors)
X_k = me
X_k = X_0
Beta = 100
k = 0
check = 1000
max_iter = 20000

while check > 10**-8 and k < max_iter:
    
    #get next iterate from descent
    X_k1 = X_k - (1/Beta) * grad(X_k)
    
    #project back into cone with eigen decomp
    [eigenvalues, V] = np.linalg.eig(X_k1)
    eigenvalues[eigenvalues < 0] = 0
    Lambda = np.diag(eigenvalues)
    X_k1 = np.dot(np.dot(V, Lambda), np.transpose(V))
    
    #set up for next iteration
    check = np.linalg.norm(X_k1 - X_k)  
    X_k = X_k1
    
    if(k%1000 == 0):
        print(k)
    k = k+1



0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000


In [25]:
print("Real sensors: ")
print(sensors)
print("Generated Sensors: ")
generated = np.transpose(X_k[0:dim, dim:dim+num_sensors])
correct = np.transpose(me[0:dim, dim:dim+num_sensors])
print(generated)
print("Correct: ")
print(correct)

Real sensors: 
[[ 1.54156573 -1.76389547]
 [-1.05057748  1.03173858]
 [ 1.98800959  1.01284084]
 [-1.45969671 -1.38504371]
 [-1.77460924  1.13546463]
 [ 0.63673365 -0.58291593]
 [-0.88334992 -1.69480079]
 [ 0.50113216 -0.31842127]
 [-1.25266142 -0.27797936]
 [-1.51687772 -0.96143237]]
Generated Sensors: 
[[ 1.54731918 -1.77165812]
 [-1.04461862  1.02349457]
 [ 1.99247072  1.00455557]
 [-1.4528653  -1.39260742]
 [-1.76840086  1.12722361]
 [ 0.64240995 -0.59080108]
 [-0.87669238 -1.70231672]
 [ 0.5067707  -0.32635264]
 [-1.24625099 -0.28592622]
 [-1.51014966 -0.96914022]]
Correct: 
[[ 1.54156573 -1.76389547]
 [-1.05057748  1.03173858]
 [ 1.98800959  1.01284084]
 [-1.45969671 -1.38504371]
 [-1.77460924  1.13546463]
 [ 0.63673365 -0.58291593]
 [-0.88334992 -1.69480079]
 [ 0.50113216 -0.31842127]
 [-1.25266142 -0.27797936]
 [-1.51687772 -0.96143237]]


In [259]:
def ldl_decomp(A):
    A = np.matrix(A)

    S = np.diag(np.diag(A))
    Sinv = np.diag(1/np.diag(A))
    D = np.matrix(S.dot(S))
    Lch = np.linalg.cholesky(A)
    L = np.matrix(Lch.dot(Sinv))
    return L, D

#prepare iterations
thing = random.rand(dim+num_sensors,dim+num_sensors)
X_0 = np.dot(thing,thing.transpose())
X_0 = np.eye(dim+num_sensors)
X_k = X_0
Beta = 100
k = 0
check = 1000
max_iter = 20000

    
while check > 10**-8 and k < max_iter:
    
    #get next iterate from descent
    X_k1 = X_k - (1/Beta) * grad(X_k)
    
    #project back into cone with eigen decomp
    [L, D] = ldl_decomp(X_k1)
    D[D < 0] = 0
    D_psd = np.diag(D)
    X_k1 = np.dot(np.dot(L, D), np.transpose(L))
    
    #set up for next iteration
    check = np.linalg.norm(X_k1 - X_k)  
    X_k = X_k1
    
    if(k%1000 == 0):
        print(k)
    k = k+1

0
1000
2000
3000
4000
5000


LinAlgError: Matrix is not positive definite

In [212]:
me = np.zeros((dim+num_sensors,dim+num_sensors))
me[0:dim,0:dim] = np.eye(dim)
me[0:dim, dim:dim+num_sensors] = np.transpose(sensors)
me[dim:dim+num_sensors, 0:dim] = sensors
me[num_sensors:dim+num_sensors,num_sensors:dim+num_sensors] = \
        np.dot(np.transpose(sensors),sensors)

make_AX_less_b(me)

[0.0,
 0.0,
 0.0,
 -6.0543978243369905,
 -0.4254131200884172,
 -3.6293859911697908,
 -4.1914071380431404,
 -3.2186659061052785,
 -6.8219613854772669,
 -2.3777212072599072,
 5.9167859115221191,
 16.421254501737202,
 -3.2818452180582298,
 -13.286462951137706,
 -2.0033990664612293,
 -2.8436844382657802,
 -0.033661863263293165,
 -8.9766942833819314,
 5.3181400185813379,
 5.4121199770106667,
 -5.2026794839133785,
 -2.3085081291098395,
 -1.8087626736644054,
 -3.8436493537534271,
 -2.7752121597964439,
 6.7977534793777057,
 14.593556859355562,
 -14.414563740085788,
 -13.086033815957123,
 -14.648313468351745,
 -11.255242373876593,
 4.9009926129677863,
 17.218461074969575,
 -0.13845570996034789,
 -2.0197532330633368,
 -3.3183065697499936,
 2.6029976861337429,
 6.8808490868078316,
 -2.9587662042100917,
 -2.1034475913264417,
 2.6331062155750962,
 8.5078220775542057,
 -9.4485275071455579,
 4.6238029717387175,
 4.1648569573554965,
 -0.41121501709174524,
 11.6845926196882,
 27.52930228342689,
 -0.141

In [28]:
import scipy
from scipy.linalg import eigh
def pca(X):
    [evalues, evectors] = scipy.linalg.eigh(X)
    
    idx = evalues.argsort()[::-1]   
    evalues = evalues[idx]
    evectors = evectors[:,idx]
    
    matrix = np.zeros(X.shape)
    width = evalues.shape[0]
    for i in range(8):
        index = width - i - 1
        if index >= 0 and evalues[index] > 0:
            matrix += evalues[index] * np.outer(evectors[:, index], evectors[:, index])
    return matrix

In [29]:
thing = random.rand(dim+num_sensors,dim+num_sensors)
X_0 = np.dot(thing,thing.transpose())
X_0 = np.eye(dim+num_sensors)
X_k = X_0
Beta = 100
k = 0
check = 1000
max_iter = 20000

while check > 10**-8 and k < max_iter:
    
    #get next iterate from descent
    X_k1 = X_k - (1/Beta) * grad(X_k)
    
    #project back into cone with eigen decomp
    X_k1 = pca(X_k1) 
    
    #set up for next iteration
    check = np.linalg.norm(X_k1 - X_k)  
    X_k = X_k1
    
    if(k%1000 == 0):
        print(k)
    k = k+1

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000


In [30]:
print("Real sensors: ")
print(sensors)
print("Generated Sensors: ")
generated = np.transpose(X_k[0:dim, dim:dim+num_sensors])
correct = np.transpose(me[0:dim, dim:dim+num_sensors])
print(generated)
print("Correct: ")
print(correct)

Real sensors: 
[[ 1.54156573 -1.76389547]
 [-1.05057748  1.03173858]
 [ 1.98800959  1.01284084]
 [-1.45969671 -1.38504371]
 [-1.77460924  1.13546463]
 [ 0.63673365 -0.58291593]
 [-0.88334992 -1.69480079]
 [ 0.50113216 -0.31842127]
 [-1.25266142 -0.27797936]
 [-1.51687772 -0.96143237]]
Generated Sensors: 
[[ 0.0515302  -0.12690548]
 [-0.02343274 -0.01411685]
 [-0.23604401 -0.20466439]
 [ 0.13311265 -0.07952421]
 [ 0.03535156 -0.01629193]
 [ 0.06025325 -0.10577756]
 [-0.09492264  0.24431221]
 [ 0.19514649 -0.2408509 ]
 [ 0.18934783 -0.14848494]
 [ 0.1036183  -0.00079884]]
Correct: 
[[ 1.54156573 -1.76389547]
 [-1.05057748  1.03173858]
 [ 1.98800959  1.01284084]
 [-1.45969671 -1.38504371]
 [-1.77460924  1.13546463]
 [ 0.63673365 -0.58291593]
 [-0.88334992 -1.69480079]
 [ 0.50113216 -0.31842127]
 [-1.25266142 -0.27797936]
 [-1.51687772 -0.96143237]]
